## Лабораторная работа №2 по курсу "Машинное обучение"

Васильев Максим М8О-307Б-18, Вариант 3

### Датасет

Для начала необходимо загрузить датасет из первой лабораторной работы:

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score


data = pd.read_csv('gender_classification_v7.csv').dropna()
data

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
0,1,11.8,6.1,1,0,1,1,Male
1,0,14.0,5.4,0,0,1,0,Female
2,0,11.8,6.3,1,1,1,1,Male
3,0,14.4,6.1,0,1,1,1,Male
4,1,13.5,5.9,0,0,0,0,Female
...,...,...,...,...,...,...,...,...
4996,1,13.6,5.1,0,0,0,0,Female
4997,1,11.9,5.4,0,0,0,0,Female
4998,1,12.9,5.7,0,0,0,0,Female
4999,1,13.2,6.2,0,0,0,0,Female


In [2]:
cols = data.columns
data[cols] = data[cols].apply(lambda x: pd.Categorical(x).codes)
data

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
0,1,4,10,1,0,1,1,1
1,0,26,3,0,0,1,0,0
2,0,4,12,1,1,1,1,1
3,0,30,10,0,1,1,1,1
4,1,21,8,0,0,0,0,0
...,...,...,...,...,...,...,...,...
4996,1,22,0,0,0,0,0,0
4997,1,5,3,0,0,0,0,0
4998,1,15,6,0,0,0,0,0
4999,1,18,11,0,0,0,0,0


Как и в предыдущей лабораторной работе, нам необходимо определить пол человека:

In [3]:
X, Y = data.drop(columns=['gender']).to_numpy(), np.array(data['gender'])

In [4]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.15, random_state=15)

### Логистическая регрессия

Необходимо написать логистическую функцию:

In [5]:
from math import exp

def logist(x):
    return 1 / (1 + exp(-np.dot(x, lr_weights)))

Теперь нам нужно найти для логистической функции веса. Назовём их lr_weights.

In [6]:
lr_weights = np.zeros(Xtrain.shape[1] + 1)
lr_xtrain = np.hstack((Xtrain, np.ones((Xtrain.shape[0], 1)))) #добавляем единицы в конец каждой выборки

#процесс обучения с шагом 0.1
for repeat in range(100):
    for i in range(lr_xtrain.shape[0]):
        x = lr_xtrain[i]
        y = Ytrain[i]
        lr_weights = lr_weights + 0.05 * np.dot(y - logist(x), x)

И, наконец, проверяем, насколько точна полученная нами регрессия:

In [7]:
lr_y0 = np.zeros(Ytest.shape[0])

for i in range(Ytest.shape[0]):
    if logist(np.append(Xtest[i], 1)) < 0.5:
        lr_y0[i] = 0
    else:
        lr_y0[i] = 1

accuracy_score(Ytest, lr_y0)

0.966711051930759

Реализация sklearn

In [8]:
from sklearn.linear_model import LogisticRegression
sk_lr = LogisticRegression(max_iter=500)
sk_lr.fit(Xtrain, Ytrain)

accuracy_score(Ytest, sk_lr.predict(Xtest))

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method

0.9693741677762983

Точности поразительно схожи. И наша точность довольно высока, т.е. можно сделать вывод о том, что пол человека легко определяется простой линейной функцией от его признаков.

### Дерево решений

Для описания вершины дерева лучше всего создать класс:

In [9]:
class Node:
    def __init__(self, index, value):
        self.index = index
        self.value = value
        self.left = None
        self.right = None

По алгоритму ID3 построим дерево решений: 

In [10]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

class DecisionTree:
    class _DataPointer:
        def __init__(self, column, value):
            self.column = column
            self.value  = value

    class _Node:
        def __init__(self, false_node, true_node, pointer):
            self.false_node = false_node
            self.true_node  = true_node
            self.pointer    = pointer

    class _Leaf:
        def __init__(self, prediction):
            self.prediction = prediction

    def __init__(self):
        self._root_node = None

    def train(self, feats, labels):
        self._root_node = self._build_tree(feats, labels)
        
    def predict(self, feats):
        if self._root_node is None:
            raise RuntimeError("Model is untrained")
        predicts = []
        for feat_list in feats:
            predicts.append(self._match(feat_list, self._root_node))
        return predicts

    def _build_tree(self, feats, labels):
        info_gain, pointer = self._find_best_split(feats, labels)
        if info_gain == 0:
            return self._Leaf(np.bincount(labels).argmax())
        false_feats, false_labels, true_feats, true_labels = self._partite(feats, labels, pointer)
        false_node = self._build_tree(false_feats, false_labels)
        true_node  = self._build_tree(true_feats,  true_labels)
        return self._Node(false_node, true_node, pointer)

    def _find_best_split(self, feats, labels):
        max_info_gain = 0
        pointer = None
        current_impurity = self._gini(labels)
        for col_i, col in enumerate(feats.T):
            unique_vals = np.unique(col)
            for u_val in unique_vals :
                split_point = self._DataPointer(col_i, u_val)
                _, false_labels, _, true_labels = self._partite(feats, labels, split_point)
                if len(false_labels) == 0 or len(true_labels) == 0:
                    continue
                info_gain = self._info_gain(false_labels, true_labels, current_impurity)
                if info_gain > max_info_gain:
                    max_info_gain = info_gain
                    pointer = split_point
        return max_info_gain, pointer

    def _partite(self, feats, labels, split_point):
        column = split_point.column
        value  = split_point.value
        false_indecies = feats[:, column] <  value
        true_indecies  = feats[:, column] >= value
        false_feats = feats[false_indecies, :]
        true_feats  = feats[true_indecies, :]
        false_labels = labels[false_indecies]
        true_labels  = labels[true_indecies]
        return false_feats, false_labels, true_feats, true_labels
        
    def _match(self, feats, node):
        if isinstance(node, self._Leaf):
            return node.prediction
        to_true = feats[node.pointer.column] >= node.pointer.value
        if to_true:
            return self._match(feats, node.true_node)
        else:
            return self._match(feats, node.false_node)

    def _info_gain(self, false_labels, true_labels, parent_impurity):
        false_c = len(false_labels)
        true_c  = len(true_labels)
        total = true_c + false_c
        false_gini = self._gini(false_labels) * (false_c / total)
        true_gini  = self._gini(true_labels) * (true_c / total)
        return parent_impurity - false_gini - true_gini

    def _gini(self, classes):
        probs = np.bincount(classes) / len(classes)
        return 1 - np.sum(probs ** 2)

In [11]:
# моя реализация
custom_tree = DecisionTree()
custom_tree.train(Xtrain, Ytrain)
accuracy_score(Ytest, custom_tree.predict(Xtest))

0.966711051930759

Реализация sklearn

In [12]:
# sklearn
from sklearn.tree import DecisionTreeClassifier
sk_tree = DecisionTreeClassifier()
sk_tree.fit(Xtrain, Ytrain)

accuracy_score(Ytest, sk_tree.predict(Xtest))

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidan

0.9640479360852197

В моей реализации точность оказалась повыше, но программа работала чуть дольше

### RANDOM FOREST



Если не ограничивать глубину деревьев решений, то они будут стремиться максимально делить тестовые данные, в плоть до максимально чистых листьев. Random Forest стремится решить эту проблему: вместо одного дерева будет создаваться лес, причем эти деревья будет тренироваться случайно - из входных пар (Особенности, класс) $(X_1, y_1), (X_2, y_2), ..., (X_n, y_n)$ ($n$ пар) для каждого дерева будет создаваться независимая случайная выборка с заменами (пары могут повторяться многократно) и дерево будет тренироваться на ней.

При обучении каждое дерево будет возвращать свой вариант классификации, после чего результатом будет наиболее частый класс (или среднее значение в случае регрессии).

In [13]:
class RandomForest:
    def __init__(self, n_trees):
        self._trees = [DecisionTree() for _ in range(n_trees)]

    def train(self, feats, labels):
        sample_size = len(feats)
        for tree in self._trees:
            sorted_idxs = range(sample_size)
            sample_idxs = np.random.choice(sorted_idxs, sample_size, replace=True)
            feats_sample = feats[sample_idxs, :]
            labels_sample = labels[sample_idxs]
            tree.train(feats_sample, labels_sample)

    def predict(self, feats):
        def find_most_common(row):
            return np.bincount(row).argmax()
        predicts = []
        for tree in self._trees:
            predicts.append(tree.predict(feats))
        votes_per_sample = np.array(predicts).T
        votes = np.apply_along_axis(find_most_common, 1, votes_per_sample)
        return votes.flatten()

In [14]:
# моя реализация
custom_forest = RandomForest(10)
custom_forest.train(Xtrain, Ytrain)
accuracy_score(Ytest, custom_forest.predict(Xtest))

0.9733688415446072

Реализация sklearn

In [15]:
# sklearn
from sklearn.ensemble import RandomForestClassifier
rf_tree = RandomForestClassifier(10)
rf_tree.fit(Xtrain, Ytrain)
accuracy_score(Ytest, rf_tree.predict(Xtest))

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:32: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ._gradient_boosting import predict_stages
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:32: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ._gradient_boost

0.9760319573901465

Точность с sklearn практиечски одинаковая

### Вывод

Все реализации показали достаточно высокую точность, но это все скорее всего из-за простейшего датасета. Мои реализации практиечски не отставали от реалзиазций sklearn, так что я доволен результатом.